<a href="https://colab.research.google.com/github/AslauAlexandru/quantum_machine_learning/blob/main/Quantum_Support_Vector_Machines_(_QSVM_)_using_Qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Support Vector Machines ( QSVM ) using Qiskit

## Installing Dependencies

In [1]:
"""
!uv pip install scikit-learn
#!uv pip install scikit-learn==1.2.1
!uv pip install pennylane==0.26
!uv pip install qiskit==0.39.2
!uv pip install qiskit_machine_learning==0.5.0
"""

'\n!uv pip install scikit-learn\n#!uv pip install scikit-learn==1.2.1\n!uv pip install pennylane==0.26\n!uv pip install qiskit==0.39.2\n!uv pip install qiskit_machine_learning==0.5.0\n'

In [2]:
#!pip uninstall numpy -y
!uv pip install scikit-learn pennylane qiskit qiskit_machine_learning numpy

Using Python 3.11.13 environment at: /usr
Audited 5 packages in 250ms


## Setting Up Data and Preprocessing

In [3]:
# Quantum Support Vector Machines ( QSVM ) using Qiskit.:
# https://medium.com/@devmallyakarar/quantum-support-vector-machines-qsvm-using-qiskit-eee347e81d83

import numpy as np
seed = 1234
np.random.seed(seed)

from sklearn.datasets import load_wine
x, y = load_wine(return_X_y=True)

x = x[:59+71]
y = y[:59+71]

from sklearn.model_selection import train_test_split
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size=0.9)

from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)

##  Quantum Kernel and Support Vector Machine

In [4]:
import pennylane as qml

nqubits = 4
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires=range(nqubits))

from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel=qkernel).fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(svm.predict(x_test), y_test))

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


Accuracy: 0.9230769230769231


## PCA for Dimensionality Reduction

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

nqubits = 8
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires=range(nqubits))

svm = SVC(kernel=qkernel).fit(xs_tr, y_tr)

print("Accuracy:", accuracy_score(svm.predict(xs_test), y_test))

Accuracy: 1.0


##  ZZFeatureMap and SVM with Quantum Kernel

In [6]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):
    nload = min(len(data), nqubits)
    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires=i)
    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]
        qml.CZ(wires=[q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) * (np.pi - data[q1]), wires=q1)
        qml.CZ(wires=[q0, q1])

nqubits = 4
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    ZZFeatureMap(nqubits, a)
    qml.adjoint(ZZFeatureMap)(nqubits, b)
    return qml.probs(wires=range(nqubits))

svm = SVC(kernel=qkernel).fit(xs_tr, y_tr)
print("Accuracy:", accuracy_score(svm.predict(xs_test), y_test))

Accuracy: 0.8461538461538461


## Using Qiskit for Quantum SVM

In [7]:
#!pip uninstall qiskit qiskit_machine_learning qiskit_aer -y
#!uv pip install qiskit==0.45.0 qiskit_machine_learning==0.7.0 qiskit_aer==0.13.0

In [8]:
from qiskit import *
from qiskit.circuit import Parameter
parameter = Parameter("x")
qc = QuantumCircuit(1)
qc.rx(parameter, 0)

from qiskit.circuit import ParameterVector
parameter = ParameterVector("x", length=2)
qc = QuantumCircuit(2)
qc.rx(parameter[0], 0)
qc.rx(parameter[1], 1)

from qiskit.circuit.library import ZZFeatureMap

zzfm = ZZFeatureMap(8)

"""
# You need to try this to verify FidelityQuantumKernel as QuantumKernel or QuantumKernel,
# because depends on qiskit_machine_learning.kernels version
# Attempting an alternative import path for QuantumKernel
try:
    from qiskit_machine_learning.kernels import QuantumKernel
except ImportError:
    # Trying a common alternative/successor class
    from qiskit_machine_learning.kernels import FidelityQuantumKernel as QuantumKernel
"""

from qiskit_machine_learning.kernels import FidelityQuantumKernel as QuantumKernel

from qiskit.providers.aer import AerSimulator

# Modified to remove the quantum_instance argument
qkernel = QuantumKernel(feature_map=zzfm)

from qiskit_machine_learning.algorithms import QSVC

# In newer versions, the quantum instance might be passed to QSVC or configured differently
# If this still fails, we may need to explicitly pass a backend to QSVC or update imports further
qsvm = QSVC(quantum_kernel=qkernel)
qsvm.fit(xs_tr, y_tr)

print("Accuracy:", accuracy_score(qsvm.predict(xs_test), y_test))

Accuracy: 0.7692307692307693
